In [1]:


# things we need for NLP
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random



In [2]:


# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

#a=intents['intents']



In [3]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

27 documents
9 classes ['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']
48 unique stemmed words ["'d", "'s", 'a', 'acceiv', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'do', 'doe', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'lat', 'lik', 'mastercard', 'mop', 'of', 'on', 'op', 'rent', 'see', 'tak', 'thank', 'that', 'ther', 'thi', 'to', 'today', 'we', 'what', 'when', 'which', 'work', 'yo', 'you']


In [4]:
documents



[(['Hi'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 (['Bye'], 'goodbye'),
 (['See', 'you', 'later'], 'goodbye'),
 (['Goodbye'], 'goodbye'),
 (['Thanks'], 'thanks'),
 (['Thank', 'you'], 'thanks'),
 (['That', "'s", 'helpful'], 'thanks'),
 (['What', 'hours', 'are', 'you', 'open', '?'], 'hours'),
 (['What', 'are', 'your', 'hours', '?'], 'hours'),
 (['When', 'are', 'you', 'open', '?'], 'hours'),
 (['Which', 'mopeds', 'do', 'you', 'have', '?'], 'mopeds'),
 (['What', 'kinds', 'of', 'mopeds', 'are', 'there', '?'], 'mopeds'),
 (['What', 'do', 'you', 'rent', '?'], 'mopeds'),
 (['Do', 'you', 'take', 'credit', 'cards', '?'], 'payments'),
 (['Do', 'you', 'accept', 'Mastercard', '?'], 'payments'),
 (['Are', 'you', 'cash', 'only', '?'], 'payments'),
 (['Are', 'you', 'open', 'today', '?'], 'opentoday'),
 (['When', 'do', 'you', 'open', 'today', '?'], 'opentoday'),
 (['What', 'are', 'your',

In [5]:
words

["'d",
 "'s",
 'a',
 'acceiv',
 'anyon',
 'ar',
 'bye',
 'can',
 'card',
 'cash',
 'credit',
 'day',
 'do',
 'doe',
 'good',
 'goodby',
 'hav',
 'hello',
 'help',
 'hi',
 'hour',
 'how',
 'i',
 'is',
 'kind',
 'lat',
 'lik',
 'mastercard',
 'mop',
 'of',
 'on',
 'op',
 'rent',
 'see',
 'tak',
 'thank',
 'that',
 'ther',
 'thi',
 'to',
 'today',
 'we',
 'what',
 'when',
 'which',
 'work',
 'yo',
 'you']

In [6]:
classes

['goodbye',
 'greeting',
 'hours',
 'mopeds',
 'opentoday',
 'payments',
 'rental',
 'thanks',
 'today']

In [7]:


# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])



In [8]:
len(train_y[1])

9

In [9]:


# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')



Training Step: 3999  | total loss: 0.12745 | time: 0.012s
| Adam | epoch: 1000 | loss: 0.12745 - acc: 0.9986 -- iter: 24/27
Training Step: 4000  | total loss: 0.12682 | time: 0.015s
| Adam | epoch: 1000 | loss: 0.12682 - acc: 0.9988 -- iter: 27/27
--
INFO:tensorflow:/home/rajat/Documents/chatbot/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [10]:


def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

p = bow("is your shop open today?", words)
print (p)
print (classes)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 1 0]
['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']


In [11]:
print(model.predict([p]))


# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )



[[  6.20139716e-03   9.11312497e-07   2.09380127e-03   8.72135875e-09
    6.34144664e-01   1.57941940e-06   1.25785232e-06   2.78727763e-04
    3.57277662e-01]]


In [12]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
intents = json.load(json_data)

# load our saved model
model.load('./model.tflearn')



INFO:tensorflow:Restoring parameters from /home/rajat/Documents/chatbot/model.tflearn


In [32]:
results=model.predict([bow("is your shop open today?", words)])[0]
results

array([  6.20139716e-03,   9.11312497e-07,   2.09380127e-03,
         8.72135875e-09,   6.34144664e-01,   1.57941940e-06,
         1.25785232e-06,   2.78727763e-04,   3.57277662e-01], dtype=float32)

In [33]:
results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]


In [34]:
results

[[4, 0.63414466], [8, 0.35727766]]

In [35]:
results.sort(key=lambda x: x[1], reverse=True)

In [36]:
results

[[4, 0.63414466], [8, 0.35727766]]

In [13]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [22]:
classify('is your shop open today?')

[('opentoday', 0.63414466), ('today', 0.35727766)]

In [38]:
response('hello')

Hello, thanks for visiting


In [45]:
response('is your shop open today')

We're open every day from 9am-9pm


In [46]:
response('I want to book a moped')

Are you looking to rent today or later this week?


In [41]:
response('today')

For rentals today please call 1-800-MYMOPED


In [44]:
response('later')

See you later, thanks for visiting


In [37]:






# clear context
response("Bye", show_details=True)

response("thanks, your great")

tag: goodbye
Bye! Come back again soon.
Any time!
